In [10]:
# ==================
# Library
# ==================
import warnings
warnings.simplefilter('ignore')
import math
import pandas as pd
import numpy as np
import sys
import time
import datetime
from contextlib import contextmanager
import logging
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score,average_precision_score
from sklearn.model_selection import StratifiedKFold, KFold,GroupKFold,StratifiedGroupKFold
import torch
import torch.nn as nn
from torch.nn import TransformerEncoder
from torch.nn import TransformerDecoder
from torch.nn import LayerNorm
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.optim import lr_scheduler
from transformers import AdamW, get_linear_schedule_with_warmup
import gc
import random
import os
%matplotlib inline
sys.path.append("../src/")
from logger import setup_logger, LOGGER
from util_tool import reduce_mem_usage
pd.set_option('display.max_columns', 300)

In [11]:
# ===================
# Settings
# ===================
debug = False
ex = "153"
if not os.path.exists(f"../output/exp/ex{ex}"):
    os.makedirs(f"../output/exp/ex{ex}")
    os.makedirs(f"../output/exp/ex{ex}/ex{ex}_model")
logger_path = f"../output/exp/ex{ex}/ex_{ex}.txt"
model_path =f"../output/exp/ex{ex}/ex{ex}_model/ex{ex}.pth"
# config
seed = 0
shuffle = True
n_splits = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model config
batch_size = 24
n_epochs = 15
lr = 1e-3
weight_decay = 0.05
num_warmup_steps = 10

In [12]:
# =====================
# data
# =====================
seq_len = 15000
id_path = f"../output/fe/fe074/fe074_id.parquet"
numerical_path = f"../output/fe/fe074/fe074_num_array.npy"
target_path = f"../output/fe/fe074/fe074_target_array.npy"
mask_path = f"../output/fe/fe074/fe074_mask_array.npy"
valid_path = f"../output/fe/fe074/fe074_valid_array.npy"
pred_use_path = f"../output/fe/fe074/fe074_pred_use_array.npy"
time_path = f"../output/fe/fe074/fe074_time_array.npy"

In [13]:
# ====================
# Function
# ====================
def preprocess(numerical_array, 
               mask_array,
               valid_array,
               ):
    
    attention_mask = mask_array == 0

    return {
        'input_data_numerical_array': numerical_array,
        'input_data_mask_array': mask_array,
        'input_data_valid_array': valid_array,
        'attention_mask': attention_mask,
    }

class FogDataset(Dataset):
    def __init__(self, numerical_array, 
                 mask_array,valid_array,
                 train = True, y = None):
        self.numerical_array = numerical_array
        self.mask_array = mask_array
        self.valid_array = valid_array
        self.train = train
        self.y = y
    
    def __len__(self):
        return len(self.numerical_array)

    def __getitem__(self, item):
        data = preprocess(
            self.numerical_array[item],
            self.mask_array[item],
            self.valid_array[item], 
            
        )

        # Return the processed data where the lists are converted to `torch.tensor`s
        if self.train : 
            return {
              'input_data_numerical_array': torch.tensor(data['input_data_numerical_array'],dtype=torch.float32),
              'input_data_mask_array':torch.tensor(data['input_data_mask_array'], dtype=torch.long),  
              'input_data_valid_array':torch.tensor(data['input_data_valid_array'], dtype=torch.long),   
              'attention_mask': torch.tensor(data["attention_mask"], dtype=torch.bool),
              "y":torch.tensor(self.y[item], dtype=torch.float32)
               }
        else:
            return {
             'input_data_numerical_array': torch.tensor(data['input_data_numerical_array'],dtype=torch.float32),
              'input_data_mask_array':torch.tensor(data['input_data_mask_array'], dtype=torch.long),  
              'input_data_valid_array':torch.tensor(data['input_data_valid_array'], dtype=torch.long),  
              'attention_mask': torch.tensor(data["attention_mask"], dtype=torch.bool),
               }

In [14]:
class FogRnnModel(nn.Module):
    def __init__(
        self, dropout=0.2,
        input_numerical_size=9,
        numeraical_linear_size = 64,
        model_size = 128,
        linear_out = 128,
        out_size=3):
        super(FogRnnModel, self).__init__()
        self.numerical_linear  = nn.Sequential(
                nn.Linear(input_numerical_size, numeraical_linear_size),
                nn.LayerNorm(numeraical_linear_size)
            )
        
        self.rnn = nn.GRU(numeraical_linear_size, model_size,
                            num_layers = 2, 
                            batch_first=True,
                            bidirectional=True)
        self.linear_out  = nn.Sequential(
                nn.Linear(model_size*2, 
                          linear_out),
                nn.LayerNorm(linear_out),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(linear_out, 
                          out_size))
        self._reinitialize()
        
    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        for name, p in self.named_parameters():
            if 'rnn' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
    
    def forward(self, numerical_array,
                mask_array,
                attention_mask):
        
        numerical_embedding = self.numerical_linear(numerical_array)
        output,_ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        return output
    
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [15]:
@contextmanager
def timer(name):
    t0 = time.time()
    yield 
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')
    
setup_logger(out_file=logger_path)

2023-05-29 11:51:00,533 - INFO - logger set up


<RootLogger root (DEBUG)>

In [16]:
# ====================
# main
# ====================
df_id = pd.read_parquet(id_path)
numerical_array = np.load(numerical_path)
target_array = np.load(target_path)
mask_array = np.load(mask_path)
valid_array = np.load(valid_path)
pred_use_array = np.load(pred_use_path)
time_array = np.load(time_path)

In [17]:
with timer("gru"):
    set_seed(seed)
    for fold in range(5):
        with timer(f"fold {fold}"):
            val_numerical_array = numerical_array.copy()
            val_target_array = target_array.copy()
            val_mask_array = mask_array.copy()
            val_valid_array = valid_array.copy()
            val_pred_array = pred_use_array.copy()

            val_ = FogDataset(val_numerical_array,
                                val_mask_array,
                                val_valid_array, 
                                train=True,
                                y=val_target_array)

            val_loader = DataLoader(dataset=val_, 
                                batch_size=batch_size, 
                                shuffle = False , num_workers=8)

            model = FogRnnModel()
            model.load_state_dict(torch.load(f"../output/exp/ex{ex}/ex{ex}_model/ex{ex}_{fold}.pth"))
            model = model.to(device)
            model.eval()  # switch model to the evaluation mode
            val_preds = np.ndarray((0,seq_len,3))
            tk0 = tqdm(val_loader, total=len(val_loader))
            with torch.no_grad():  # Do not calculate gradient since we are only predicting
                # Predicting on validation set
                for d in tk0:
                    input_data_numerical_array = d['input_data_numerical_array'].to(device)
                    input_data_mask_array = d['input_data_mask_array'].to(device)
                    attention_mask = d['attention_mask'].to(device)
                    output = model(input_data_numerical_array, 
                               input_data_mask_array,
                               attention_mask)
                    val_preds = np.concatenate([val_preds, output.sigmoid().detach().cpu().numpy()], axis=0)
            np.save(f"../output/exp/ex{ex}/ex{ex}_notype_{fold}_oof_{seq_len}.npy",val_preds)

  0%|          | 0/57 [00:00<?, ?it/s]

2023-05-29 11:51:45,303 - INFO - [fold 0] done in 44 s


  0%|          | 0/57 [00:00<?, ?it/s]

2023-05-29 11:52:28,760 - INFO - [fold 1] done in 43 s


  0%|          | 0/57 [00:00<?, ?it/s]

2023-05-29 11:53:11,524 - INFO - [fold 2] done in 43 s


  0%|          | 0/57 [00:00<?, ?it/s]

2023-05-29 11:53:52,114 - INFO - [fold 3] done in 41 s


  0%|          | 0/57 [00:00<?, ?it/s]

2023-05-29 11:54:35,767 - INFO - [fold 4] done in 44 s
2023-05-29 11:54:35,769 - INFO - [gru] done in 214 s


In [18]:
id_array = df_id["Id"].values
for i in range(5):
    pred_all = []
    pred = np.load(f"../output/exp/ex{ex}/ex{ex}_notype_{i}_oof_{seq_len}.npy")
    for v in tqdm(range(len(pred_use_array))):
        use_ = pred_use_array[v,:] == 1
        pred_ = pred[v,use_ == 1,:]
        time_ = time_array[v,use_ == 1]
        Id = id_array[v]
        pred_df = pd.DataFrame()
        pred_df["Time"] = time_
        pred_df["Id"] = Id
        pred_df["StartHesitation"] = pred_[:,0]
        pred_df["Turn"] = pred_[:,1]
        pred_df["Walking"] = pred_[:,2]
        pred_all.append(pred_df)
    pred_all = pd.concat(pred_all).reset_index(drop=True)
    pred_all.to_parquet(f"../output/exp/ex{ex}/ex{ex}_notype_{i}_pred_{seq_len}.parquet")

  0%|          | 0/1345 [00:00<?, ?it/s]

  0%|          | 0/1345 [00:00<?, ?it/s]

  0%|          | 0/1345 [00:00<?, ?it/s]

  0%|          | 0/1345 [00:00<?, ?it/s]

  0%|          | 0/1345 [00:00<?, ?it/s]